Analyze spin and bond angles of KGA-relevant subset: connected TM octahedra

In [1]:
import json
from monty.json import MontyDecoder
import plotly.graph_objects as go
import plotly.express as px
import os
from pymatgen.core import Element


from utils_kga.statistical_analysis.get_spin_and_bond_angle_statistics import *
from utils_kga.statistical_analysis.ks_test import compute_ks_weighted
from utils_kga.general import pretty_plot

In [2]:
# Load edge-df
with open("data/dfs_of_magnetic_edge_information.json") as f:
    dict_all_stats = json.load(f)
all_stats = {key: pd.DataFrame.from_dict(df) for key, df in dict_all_stats.items()}

# For metadata filtering and computation of occurrences in magnetic primitive cells
with open("../../data_retrieval_and_preprocessing_MAGNDATA/data/df_grouped_and_chosen_commensurate_MAGNDATA.json") as f:
    df = json.load(f, cls=MontyDecoder)

In [3]:
# Add is_tm bool for later easier analysis
for ang_df in all_stats.values():
    ang_df["site_is_tm"] = ang_df["site_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["site_to_is_tm"] = ang_df["site_to_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["ligand_el_set"] = ang_df["ligand_elements"].apply(lambda ls: set(ls))

In [4]:
description = [
    "all edges with TM octahedra at both nodes",
    "all oxygen edges with TM octahedra at both nodes",
]
plot_dir = "plots/TM_octahedra_analysis/"
os.makedirs(plot_dir, exist_ok=True)
one_d_hist_file = "1d_histograms_bond_angle_occurrences_as_f_spin_angle.html"
one_d_hist_inverse_file = "1d_histograms_spin_angle_occurrences_as_f_bond_angle.html"
two_d_hist_file = "2d_histograms_bond_and_spin_angle_occurrences.html"

In [5]:
# Plot histograms of bond angles as f(angle between magnetic vectors) and vice versa in- and excluding ligand mult., in normalized and absolute occurrences and both for connected TM octahedra and oxygen-connected TM octahedra
write_mode = "w"
for normalize_bool, normalize_string in zip([True, False], ["normalized occurrences", "absolute occurrences"]):
    for ligand_multiplicity_bool, ligand_multiplicity_string in zip([True, False], ["ligand multiplicity included", "no ligand multiplicity included"]):
        for data_string in description:
            entries = 0
            all_spin_occus = []
            for md_id, ang_df in all_stats.items():
                subdf = ang_df.loc[(ang_df["site_ce"]=="O:6") 
                       & (ang_df["site_to_ce"]=="O:6")
                       & (ang_df["site_is_tm"]) 
                       & (ang_df["site_to_is_tm"])
                ]
                if "oxygen" in data_string:
                    subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
                if not subdf.empty:
                    entries += 1
                    n_lattice_points = df.at[md_id, "n_lattice_points"]
                    occus = get_bond_angle_occurrences(df=subdf,
                                                                     include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                     normalize=normalize_bool,
                                                                     n_lattice_points=n_lattice_points,
                                                                     spin_angle_round=-1,  # for overview plots
                                                                     bond_angle_round=7)
                    all_spin_occus.extend(occus)
            all_spin_occus_df = pd.DataFrame(columns=["spin_angle", "bond_angle", "occurrence"], data=all_spin_occus)

            # Create and save one-dimensional histograms of bond angle occurrences as f(spin angle)
            one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (deg)"),
                                                   yaxis=go.layout.YAxis(title="Occurrence"),
                                                   title=f"{data_string} ({entries} structures, {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}"))
            
            for spin_ang_lim in sorted(set(all_spin_occus_df["spin_angle"].values)):
                sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] == spin_ang_lim]
                
                one_d_fig.add_trace(go.Histogram(
                    histfunc="sum", 
                    x=sub_df["bond_angle"].values, 
                    y=sub_df["occurrence"].values, 
                    nbinsx=181,
                    name=spin_ang_lim
                ))
            one_d_fig = pretty_plot(one_d_fig)
            one_d_fig.update_layout(legend_title_text="Spin angle (deg)")
            one_d_fig.update_layout(barmode="overlay")
            one_d_fig.update_traces(opacity=0.75)
            with open(os.path.join(plot_dir, one_d_hist_file), write_mode) as f:
                f.write(one_d_fig.to_html(full_html=False, include_plotlyjs="cdn"))

            # Create and save one-dimensional histograms of spin angle occurrences as f(bond angle)
            one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Spin angle (deg)"),
                                                   yaxis=go.layout.YAxis(title="Occurrence"),
                                                   colorway=px.colors.qualitative.Alphabet,
                                                   title=f"{data_string} ({entries} structures, {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}"
                                                  ),
                                 )
            
            for bond_ang in sorted(set([round(ba, -1) for ba in all_spin_occus_df["bond_angle"].values])):
                sub_df = all_spin_occus_df.loc[round(all_spin_occus_df["bond_angle"], -1) == bond_ang]
                
                one_d_fig.add_trace(go.Histogram(
                    histfunc="sum", 
                    x=sub_df["spin_angle"].values, 
                    y=sub_df["occurrence"].values, 
                    nbinsx=19,
                    name=bond_ang
                ))
            one_d_fig = pretty_plot(one_d_fig)
            one_d_fig.update_layout(legend_title_text="Bond angle (deg)")
            one_d_fig.update_traces(opacity=0.75)
            with open(os.path.join(plot_dir, one_d_hist_inverse_file), write_mode) as f:
                f.write(one_d_fig.to_html(full_html=False, include_plotlyjs="cdn"))
            
            # Create and save 2D histogram of spin and bond angle occurrences
            two_d_fig = go.Figure(data=go.Histogram2d(
                x=all_spin_occus_df["bond_angle"].values,
                y=all_spin_occus_df["spin_angle"].values,
                z=all_spin_occus_df["occurrence"].values,
                histfunc="sum",  
                nbinsx=181, 
                nbinsy=19
                ),
                layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (deg)"),
                                 yaxis=go.layout.YAxis(title="Spin angle (deg)"),
                                 title=f"{data_string} ({entries} structures, {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}")
            )
            with open(os.path.join(plot_dir, two_d_hist_file), write_mode) as f:
                f.write(two_d_fig.to_html(full_html=False, include_plotlyjs="cdn"))

            write_mode = "a"

In [6]:
# Save special cases to pdf (FM and AFM subset, 10° angle tol for magnetic vectors), print out additional info on bond angle distributions mentioned in paper
for ligand_multiplicity_bool, ligand_multiplicity_string in zip([True, False], ["ligand multiplicity included", "no ligand multiplicity included"]):
    for normalize_bool, normalize_string in zip([True, False], ["normalized occurrences", "absolute occurrences"]):
        for data_string in description:
            print("--------------------------------------------------------------------")
            print(normalize_string, ligand_multiplicity_string, data_string)
            print("--------------------------------------------------------------------")
            entries = 0
            almost_collinear_entries = 0
            all_spin_occus = []
            for md_id, ang_df in all_stats.items():
                subdf = ang_df.loc[(ang_df["site_ce"]=="O:6")
                       & (ang_df["site_to_ce"]=="O:6")
                       & (ang_df["site_is_tm"])
                       & (ang_df["site_to_is_tm"])
                ]
                if "oxygen" in data_string:
                    subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
                if not subdf.empty:
                    entries += 1
                    n_lattice_points = df.at[md_id, "n_lattice_points"]
                    occus = get_bond_angle_occurrences(df=subdf,
                                                                     include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                     normalize=normalize_bool,
                                                                     n_lattice_points=n_lattice_points,
                                                                     spin_angle_round=0,
                                                                     bond_angle_round=7)
                    all_spin_occus.extend(occus)
                    if any([True for e in occus if (e[0] >= 170.0 or e[0] <= 10.0)]):
                        almost_collinear_entries += 1
            all_spin_occus_df = pd.DataFrame(columns=["spin_angle", "bond_angle", "occurrence"], data=all_spin_occus)

            # Percentage of almost collinear interactions in whole dataset
            afm_i =  all_spin_occus_df.loc[(all_spin_occus_df["spin_angle"] >= 170.0)]["occurrence"].values.sum()
            fm_i =  all_spin_occus_df.loc[(all_spin_occus_df["spin_angle"] <= 10.0)]["occurrence"].values.sum()
            print(f"Fraction of interactions that are collinear within 10° angle tol: {(afm_i+fm_i) / all_spin_occus_df.occurrence.values.sum()}")

            # Count percentage of almost collinear interactions with bond angles > x that are AFM
            for ba in [100, 110, 120]:
                afm_l = all_spin_occus_df.loc[(all_spin_occus_df["bond_angle"] > ba)
                                             &(all_spin_occus_df["spin_angle"] >= 170.0) ]["occurrence"].values.sum()
                fm_l = all_spin_occus_df.loc[(all_spin_occus_df["bond_angle"] > ba)
                                             &(all_spin_occus_df["spin_angle"] <= 10.0) ]["occurrence"].values.sum()
                print(f"Fraction of collinear interactions above {ba}° that are AFM: {afm_l / (afm_l + fm_l)}")


            for spin_ang_lim in [10.0, 170.0]:
                if spin_ang_lim == 10.0:
                    sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] <= spin_ang_lim]
                else:
                    sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] >= spin_ang_lim]

                spin_ang_lim_string = f">=_{spin_ang_lim}_spin_angle" if spin_ang_lim == 170.0 else f"<=_{spin_ang_lim}_spin_angle"

                # Print fraction of bond angles > 110°
                print(data_string,
                    spin_ang_lim_string,
                      "Fraction of bond angles above 110°: ",
                      sub_df.loc[sub_df["bond_angle"] > 110.0]["occurrence"].values.sum() /
                      sub_df.loc[sub_df["bond_angle"] > 0.0]["occurrence"].values.sum())
                # Print fraction of bond angles around 90°
                print(data_string,
                      spin_ang_lim_string,
                      "Fraction of bond angles between 75° and 105°: ",
                      sub_df.loc[(sub_df["bond_angle"] <= 105.0) & (sub_df["bond_angle"] >= 75.0)]["occurrence"].values.sum() /
                      sub_df.loc[sub_df["bond_angle"] > 0.0]["occurrence"].values.sum())

                one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (°)"),
                                                       yaxis=go.layout.YAxis(title="Occurrence")))

                one_d_fig.add_trace(go.Histogram(
                    histfunc="sum",
                    x=sub_df["bond_angle"].values,
                    y=sub_df["occurrence"].values,
                    nbinsx=181,
                    name=spin_ang_lim,
                    marker_color="#025268"
                ))
                one_d_fig = pretty_plot(one_d_fig)
                one_d_fig.update_layout(title=dict(
                    text=f"{data_string} ({entries} structures ({almost_collinear_entries} almost coll.), {np.sum(sub_df.occurrence.values)} / {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}, spin_ang_lim = {spin_ang_lim}°",
                font=dict(size=9)))

                if ligand_multiplicity_bool:
                    one_d_fig.update_layout(yaxis_range=[0, 590], xaxis_range=[57, 182])
                elif normalize_bool:
                    one_d_fig.update_layout(yaxis_range=[0, 17], xaxis_range=[57, 182])
                else:
                    if "oxygen" in data_string:
                        one_d_fig.update_layout(yaxis_range=[0, 280], xaxis_range=[75, 182])
                    else:
                        one_d_fig.update_layout(yaxis_range=[0, 295], xaxis_range=[57, 182])
                one_d_fig.update_yaxes(zeroline=False)
                one_d_fig.write_image(os.path.join(plot_dir, f"{data_string}_{normalize_string}_{ligand_multiplicity_string}_{spin_ang_lim_string}.pdf"))

--------------------------------------------------------------------
normalized occurrences ligand multiplicity included all edges with TM octahedra at both nodes
--------------------------------------------------------------------
Fraction of interactions that are collinear within 10° angle tol: 0.8202550232739919
Fraction of collinear interactions above 100° that are AFM: 0.7622961664884066
Fraction of collinear interactions above 110° that are AFM: 0.80787654835981
Fraction of collinear interactions above 120° that are AFM: 0.8062163439105794
all edges with TM octahedra at both nodes <=_10.0_spin_angle Fraction of bond angles above 110°:  0.2459849779909503
all edges with TM octahedra at both nodes <=_10.0_spin_angle Fraction of bond angles between 75° and 105°:  0.7107837397888945
all edges with TM octahedra at both nodes >=_170.0_spin_angle Fraction of bond angles above 110°:  0.5918159582684573
all edges with TM octahedra at both nodes >=_170.0_spin_angle Fraction of bond angles 

In [7]:
# Assert that analysis results similar upon changing spin angle tolerance to 5 °
ligand_multiplicity_bool = False
ligand_multiplicity_string = "no ligand multiplicity included"

for normalize_bool, normalize_string in zip([False], ["absolute occurrences"]):
    for data_string in description[:1]:
        print("--------------------------------------------------------------------")
        print(normalize_string, data_string)
        print("--------------------------------------------------------------------")
        entries = 0
        almost_collinear_entries = 0
        all_spin_occus = []
        for md_id, ang_df in all_stats.items():
            subdf = ang_df.loc[(ang_df["site_ce"]=="O:6")
                   & (ang_df["site_to_ce"]=="O:6")
                   & (ang_df["site_is_tm"])
                   & (ang_df["site_to_is_tm"])
            ]
            if "oxygen" in data_string:
                subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
            if not subdf.empty:
                entries += 1
                n_lattice_points = df.at[md_id, "n_lattice_points"]
                occus = get_bond_angle_occurrences(df=subdf,
                                                                 include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                 normalize=normalize_bool,
                                                                 n_lattice_points=n_lattice_points,
                                                                 spin_angle_round=0,
                                                                 bond_angle_round=7)
                all_spin_occus.extend(occus)
                if any([True for e in occus if (e[0] >= 175.0 or e[0] <= 5.0)]):
                    almost_collinear_entries += 1
        all_spin_occus_df = pd.DataFrame(columns=["spin_angle", "bond_angle", "occurrence"], data=all_spin_occus)

        # Percentage of almost collinear interactions in whole dataset
        afm_i =  all_spin_occus_df.loc[(all_spin_occus_df["spin_angle"] >= 175.0)]["occurrence"].values.sum()
        fm_i =  all_spin_occus_df.loc[(all_spin_occus_df["spin_angle"] <= 5.0)]["occurrence"].values.sum()
        print(f"Fraction of interactions that are collinear within 5° angle tol: {(afm_i+fm_i) / all_spin_occus_df.occurrence.values.sum()}")

        # Count percentage of almost collinear interactions with bond angles > x that are AFM
        for ba in [100, 110, 120]:
            afm_l = all_spin_occus_df.loc[(all_spin_occus_df["bond_angle"] > ba)
                                         &(all_spin_occus_df["spin_angle"] >= 175.0) ]["occurrence"].values.sum()
            fm_l = all_spin_occus_df.loc[(all_spin_occus_df["bond_angle"] > ba)
                                         &(all_spin_occus_df["spin_angle"] <= 5.0) ]["occurrence"].values.sum()
            print(f"Fraction of collinear interactions above {ba}° that are AFM: {afm_l / (afm_l + fm_l)}")


        for spin_ang_lim in [5.0, 175.0]:
            if spin_ang_lim == 5.0:
                sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] <= spin_ang_lim]
            else:
                sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] >= spin_ang_lim]

            spin_ang_lim_string = f">=_{spin_ang_lim}_spin_angle" if spin_ang_lim == 175.0 else f"<=_{spin_ang_lim}_spin_angle"

            # Print fraction of bond angles > 110°
            print(data_string,
                spin_ang_lim_string,
                  "Fraction of bond angles above 110°: ",
                  sub_df.loc[sub_df["bond_angle"] > 110.0]["occurrence"].values.sum() /
                  sub_df.loc[sub_df["bond_angle"] > 0.0]["occurrence"].values.sum())
            # Print fraction of bond angles around 90°
            print(data_string,
                  spin_ang_lim_string,
                  "Fraction of bond angles between 75° and 105°: ",
                  sub_df.loc[(sub_df["bond_angle"] <= 105.0) & (sub_df["bond_angle"] >= 75.0)]["occurrence"].values.sum() /
                  sub_df.loc[sub_df["bond_angle"] > 0.0]["occurrence"].values.sum())

            one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (°)"),
                                                   yaxis=go.layout.YAxis(title="Occurrence")))

            one_d_fig.add_trace(go.Histogram(
                histfunc="sum",
                x=sub_df["bond_angle"].values,
                y=sub_df["occurrence"].values,
                nbinsx=181,
                name=spin_ang_lim,
                marker_color="#025268"
            ))
            one_d_fig = pretty_plot(one_d_fig)
            one_d_fig.update_layout(title=dict(
                text=f"{data_string} ({entries} structures ({almost_collinear_entries} almost coll.), {np.sum(sub_df.occurrence.values)} / {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}, spin_ang_lim = {spin_ang_lim}°",
            font=dict(size=9)))

            if ligand_multiplicity_bool:
                one_d_fig.update_layout(yaxis_range=[0, 590], xaxis_range=[57, 182])
            elif normalize_bool:
                one_d_fig.update_layout(yaxis_range=[0, 17], xaxis_range=[57, 182])
            else:
                if "oxygen" in data_string:
                    one_d_fig.update_layout(yaxis_range=[0, 280], xaxis_range=[75, 182])
                else:
                    one_d_fig.update_layout(yaxis_range=[0, 295], xaxis_range=[57, 182])
            one_d_fig.update_yaxes(zeroline=False)
            one_d_fig.write_image(os.path.join(plot_dir, f"{data_string}_{normalize_string}_{ligand_multiplicity_string}_{spin_ang_lim_string}.pdf"))

--------------------------------------------------------------------
absolute occurrences all edges with TM octahedra at both nodes
--------------------------------------------------------------------
Fraction of interactions that are collinear within 5° angle tol: 0.7485258411377038
Fraction of collinear interactions above 100° that are AFM: 0.7185354691075515
Fraction of collinear interactions above 110° that are AFM: 0.7421875
Fraction of collinear interactions above 120° that are AFM: 0.7449664429530202
all edges with TM octahedra at both nodes <=_5.0_spin_angle Fraction of bond angles above 110°:  0.3673469387755102
all edges with TM octahedra at both nodes <=_5.0_spin_angle Fraction of bond angles between 75° and 105°:  0.600700886415172
all edges with TM octahedra at both nodes >=_175.0_spin_angle Fraction of bond angles above 110°:  0.6335679881437569
all edges with TM octahedra at both nodes >=_175.0_spin_angle Fraction of bond angles between 75° and 105°:  0.3374089168827961


In [8]:
# Assert that analysis results similar upon changing spin angle tolerance to 20 °
ligand_multiplicity_bool = False
ligand_multiplicity_string = "no ligand multiplicity included"

for normalize_bool, normalize_string in zip([False], ["absolute occurrences"]):
    for data_string in description[:1]:
        print("--------------------------------------------------------------------")
        print(normalize_string, data_string)
        print("--------------------------------------------------------------------")
        entries = 0
        almost_collinear_entries = 0
        all_spin_occus = []
        for md_id, ang_df in all_stats.items():
            subdf = ang_df.loc[(ang_df["site_ce"]=="O:6")
                   & (ang_df["site_to_ce"]=="O:6")
                   & (ang_df["site_is_tm"])
                   & (ang_df["site_to_is_tm"])
            ]
            if "oxygen" in data_string:
                subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
            if not subdf.empty:
                entries += 1
                n_lattice_points = df.at[md_id, "n_lattice_points"]
                occus = get_bond_angle_occurrences(df=subdf,
                                                                 include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                 normalize=normalize_bool,
                                                                 n_lattice_points=n_lattice_points,
                                                                 spin_angle_round=0,
                                                                 bond_angle_round=7)
                all_spin_occus.extend(occus)
                if any([True for e in occus if (e[0] >= 160.0 or e[0] <= 20.0)]):
                    almost_collinear_entries += 1
        all_spin_occus_df = pd.DataFrame(columns=["spin_angle", "bond_angle", "occurrence"], data=all_spin_occus)

        # Percentage of almost collinear interactions in whole dataset
        afm_i =  all_spin_occus_df.loc[(all_spin_occus_df["spin_angle"] >= 160.0)]["occurrence"].values.sum()
        fm_i =  all_spin_occus_df.loc[(all_spin_occus_df["spin_angle"] <= 20.0)]["occurrence"].values.sum()
        print(f"Fraction of interactions that are collinear within 20° angle tol: {(afm_i+fm_i) / all_spin_occus_df.occurrence.values.sum()}")

        # Count percentage of almost collinear interactions with bond angles > x that are AFM
        for ba in [100, 110, 120]:
            afm_l = all_spin_occus_df.loc[(all_spin_occus_df["bond_angle"] > ba)
                                         &(all_spin_occus_df["spin_angle"] >= 160.0) ]["occurrence"].values.sum()
            fm_l = all_spin_occus_df.loc[(all_spin_occus_df["bond_angle"] > ba)
                                         &(all_spin_occus_df["spin_angle"] <= 20.0) ]["occurrence"].values.sum()
            print(f"Fraction of collinear interactions above {ba}° that are AFM: {afm_l / (afm_l + fm_l)}")


        for spin_ang_lim in [20.0, 160.0]:
            if spin_ang_lim == 20.0:
                sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] <= spin_ang_lim]
            else:
                sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] >= spin_ang_lim]

            spin_ang_lim_string = f">=_{spin_ang_lim}_spin_angle" if spin_ang_lim == 160.0 else f"<=_{spin_ang_lim}_spin_angle"

            # Print fraction of bond angles > 110°
            print(data_string,
                spin_ang_lim_string,
                  "Fraction of bond angles above 110°: ",
                  sub_df.loc[sub_df["bond_angle"] > 110.0]["occurrence"].values.sum() /
                  sub_df.loc[sub_df["bond_angle"] > 0.0]["occurrence"].values.sum())
            # Print fraction of bond angles around 90°
            print(data_string,
                  spin_ang_lim_string,
                  "Fraction of bond angles between 75° and 105°: ",
                  sub_df.loc[(sub_df["bond_angle"] <= 105.0) & (sub_df["bond_angle"] >= 75.0)]["occurrence"].values.sum() /
                  sub_df.loc[sub_df["bond_angle"] > 0.0]["occurrence"].values.sum())

            one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (°)"),
                                                   yaxis=go.layout.YAxis(title="Occurrence")))

            one_d_fig.add_trace(go.Histogram(
                histfunc="sum",
                x=sub_df["bond_angle"].values,
                y=sub_df["occurrence"].values,
                nbinsx=181,
                name=spin_ang_lim,
                marker_color="#025268"
            ))
            one_d_fig = pretty_plot(one_d_fig)
            one_d_fig.update_layout(title=dict(
                text=f"{data_string} ({entries} structures ({almost_collinear_entries} almost coll.), {np.sum(sub_df.occurrence.values)} / {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}, spin_ang_lim = {spin_ang_lim}°",
            font=dict(size=9)))

            if ligand_multiplicity_bool:
                one_d_fig.update_layout(yaxis_range=[0, 590], xaxis_range=[57, 182])
            elif normalize_bool:
                one_d_fig.update_layout(yaxis_range=[0, 17], xaxis_range=[57, 182])
            else:
                if "oxygen" in data_string:
                    one_d_fig.update_layout(yaxis_range=[0, 280], xaxis_range=[75, 182])
                else:
                    one_d_fig.update_layout(yaxis_range=[0, 330], xaxis_range=[57, 182])
            one_d_fig.update_yaxes(zeroline=False)
            one_d_fig.write_image(os.path.join(plot_dir, f"{data_string}_{normalize_string}_{ligand_multiplicity_string}_{spin_ang_lim_string}.pdf"))

--------------------------------------------------------------------
absolute occurrences all edges with TM octahedra at both nodes
--------------------------------------------------------------------
Fraction of interactions that are collinear within 20° angle tol: 0.8092265001734305
Fraction of collinear interactions above 100° that are AFM: 0.7287420941672523
Fraction of collinear interactions above 110° that are AFM: 0.7544
Fraction of collinear interactions above 120° that are AFM: 0.7535714285714286
all edges with TM octahedra at both nodes <=_20.0_spin_angle Fraction of bond angles above 110°:  0.35226620768789446
all edges with TM octahedra at both nodes <=_20.0_spin_angle Fraction of bond angles between 75° and 105°:  0.6123541786192388
all edges with TM octahedra at both nodes >=_160.0_spin_angle Fraction of bond angles above 110°:  0.6452275059869996
all edges with TM octahedra at both nodes >=_160.0_spin_angle Fraction of bond angles between 75° and 105°:  0.327289314631086

In [9]:
# KS test w. 10 deg angle tol between neighboring magnetic vectors
for normalize_bool, normalize_string in zip([True, False], ["normalized occurrences", "absolute occurrences"]):
    for ligand_multiplicity_bool, ligand_multiplicity_string in zip([True, False], ["ligand multiplicity included", "no ligand multiplicity included"]):
        for data_string in description:
            afm_or_fm_entries = 0
            all_spin_occus = []
            for md_id, ang_df in all_stats.items():
                subdf = ang_df.loc[(ang_df["site_ce"]=="O:6") 
                       & (ang_df["site_to_ce"]=="O:6")
                       & (ang_df["site_is_tm"]) 
                       & (ang_df["site_to_is_tm"])
                ]
                if "oxygen" in data_string:
                    subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
                if not subdf.empty:
                    n_lattice_points = df.at[md_id, "n_lattice_points"]
                    occus = get_bond_angle_occurrences(df=subdf,
                                                                     include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                     normalize=normalize_bool,
                                                                     n_lattice_points=n_lattice_points,
                                                                     spin_angle_round=0,
                                                                     bond_angle_round=7)
                    sp_a = [ls[0] for ls in occus]
                    if any([v <= 10.0 for v in sp_a]) or any([v >= 170.0 for v in sp_a]):
                        afm_or_fm_entries += 1
                    all_spin_occus.extend(occus)
            print("------")
            print(data_string, normalize_string, ligand_multiplicity_string)
            print(f"{afm_or_fm_entries} structures with FM and / or AFM interactions (10 deg angle tol).")
            
            fm_occus = [ls for ls in all_spin_occus if ls[0] <= 10.0]
            afm_occus = [ls for ls in all_spin_occus if ls[0] >= 170.0]
            print("n_afm: ", round(sum([l[2] for l in afm_occus]), 2))
            print("n_fm: ", round(sum([l[2] for l in fm_occus]), 2))
            print("d, p: ", compute_ks_weighted(afm_occus, fm_occus))

------
all edges with TM octahedra at both nodes normalized occurrences ligand multiplicity included
351 structures with FM and / or AFM interactions (10 deg angle tol).
n_afm:  205.57
n_fm:  117.62
d, p:  (0.34647959812200463, 1.507582897805681e-08)
------
all oxygen edges with TM octahedra at both nodes normalized occurrences ligand multiplicity included
258 structures with FM and / or AFM interactions (10 deg angle tol).
n_afm:  156.04
n_fm:  81.4
d, p:  (0.3003553816463888, 9.76358314516726e-05)
------
all edges with TM octahedra at both nodes normalized occurrences no ligand multiplicity included
351 structures with FM and / or AFM interactions (10 deg angle tol).
n_afm:  208.93
n_fm:  114.1
d, p:  (0.34611554239533726, 2.007968532897115e-08)
------
all oxygen edges with TM octahedra at both nodes normalized occurrences no ligand multiplicity included
258 structures with FM and / or AFM interactions (10 deg angle tol).
n_afm:  158.51
n_fm:  78.62
d, p:  (0.2973403633132394, 0.0001